In [1]:
# In this script, we will read in CMIP5 abrupt4xCO2 and piControl air (3D) and surface (2D)
# temperature to calculate LW and SW water vapor feedbacks

# By: Ty Janoski
# Last edited: 05.29.20

In [2]:
# Import statements

import xarray as xr
import dask.array as da
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
from dask.diagnostics import ProgressBar
import xesmf as xe
import netCDF4 as nc
import glob
%matplotlib inline

In [3]:
# start by reading in the water vapor kernel
q_kern = xr.open_dataset('/dx05/tylerj/d10/Arctic_Research/CMIP5_Arctic_Amplification/\
CAM5_kernels/q.kernel.plev.nc')
lats = q_kern.lat
lons = q_kern.lon
# some models have higher vertical resolution than the kernels. We only need 17 levels
plevs = [float(p) for p in q_kern.plev]

# create a dataset with lat/lons for regridding
ds_out = xr.Dataset({'lat': (['lat'], lats),
                     'lon': (['lon'], lons),
                    }
                   )

norm_factor = io.loadmat('/dx05/tylerj/d10/Arctic_Research/CMIP5_Arctic_Amplification/'+
                         'CAM5_kernels/CAM5_qkernel_normfactor_plev_v3.mat')['dlogqdt_new']
norm_factor = np.transpose(norm_factor,(3,2,1,0))

norm_factor = da.tile(norm_factor,(150,1,1,1))
FLNT = da.tile(q_kern.FLNT,(150,1,1,1))/norm_factor
FLNTC = da.tile(q_kern.FLNTC,(150,1,1,1))/norm_factor
FSNT = da.tile(q_kern.FSNT,(150,1,1,1))/norm_factor
FSNTC = da.tile(q_kern.FSNTC,(150,1,1,1))/norm_factor

In [4]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
for mod in models:
    print(mod)
    if mod in ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2']:
        # read in air/surface temperature from last 50 years of piControl
        hus = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/hus_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).hus.isel(
            time=slice(-600,None)).sel(plev=plevs)
    else:
        path = glob.glob('/dx07/tylerj/CMIP5_output/piControl/hus_Amon_'+mod+'*.nc')[0]
        hus = xr.open_dataset(path,use_cftime=True).hus.isel(
            time=slice(-600,None)).sel(plev=plevs)
    
    # make climatology
    clim = hus.groupby(hus.time.dt.month).mean(dim='time')
    
    # read in first 150 years of abrupt4xCO2
    if mod in ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2']:
        hus = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/hus_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).hus.isel(
            time=slice(None,1800)).sel(plev=plevs)
    else:
        path = glob.glob('/dx07/tylerj/CMIP5_output/abrupt4xCO2/hus_Amon_'+mod+'*.nc')[0]
        hus = xr.open_dataset(path,use_cftime=True).hus.isel(
            time=slice(None,1800)).sel(plev=plevs)
    
    # take difference abrupt4xCO2 - climatology
    dhus = da.log(hus) - da.log(da.tile(clim,(150,1,1,1)))
    dhus = dhus.where(xr.ufuncs.isfinite(dhus),np.nan)
    
    # load in layer thickness, calculated in dz.ipynb
    dz = da.array(xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+
                           '_dz.nc',
                          use_cftime=True).dz.isel(
        time=slice(None,1800)))
    
    # regrid to CAM5 horizontal grid
    regridder = xe.Regridder(dhus,ds_out,'bilinear',periodic=True,reuse_weights=True)
    with ProgressBar():
        dq = regridder(dhus.compute()) * dz
        
    del dhus, dz, hus, clim
    
    # now we can calculate feedbacks
    LWAS = ((dq * FLNT)/-100).sum(dim='plev')
    LWCS = ((dq * FLNTC)/-100).sum(dim='plev')
    SWAS = ((dq * FSNT)/100).sum(dim='plev')
    SWCS = ((dq * FSNTC)/100).sum(dim='plev')
    del dq
    NETAS = LWAS + SWAS
    NETCS = LWCS + SWCS
    out = xr.merge([LWAS.rename('wv_LWAS'),LWCS.rename('wv_LWCS'),SWAS.rename('wv_SWAS'),
                 SWCS.rename('wv_SWCS'),NETAS.rename('wv_NETAS'),NETCS.rename('wv_NETCS')])
    out.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+
                                           mod+'_wv.nc')
    del out, LWAS, LWCS, SWAS, SWCS, NETAS, NETCS

MIROC-ESM
Reuse existing file: bilinear_64x128_192x288_peri.nc
[#                                       ] | 4% Completed |  9.6s

/home/tylerj/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[###########                             ] | 29% Completed | 19.3s

/home/tylerj/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in log
  return func(*args2)


[#############                           ] | 32% Completed | 20.6s

/home/tylerj/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: divide by zero encountered in log
  return func(*args2)


[########################################] | 100% Completed | 23.7s
